# Deep Learning Toolkit for Splunk - Zero-shot Classification Notebook

This notebook contains an example workflow how to use zero-shot classification with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

2023-10-13 07:45:01.628297: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-13 07:45:01.628335: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 2.0.3


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=2
| eval text="This is a computer system book"
| fit MLTKContainer mode=stage algo=transformers_zeroshot_classification lang=en labels=instroduction+blaming+poem prompt="This sentence is {}" from text into app:transformers_zeroshot_classification

For the input parameters, "labels" should contain all the candidate labels separated by a + symbol. The "prompt" should be a prompt string containing a pair of curly braces indicating where the label should be placed in.

In [3]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [14]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("transformers_zeroshot_classification")
print(df.describe())
print(param)

          text
count        2
unique       1
top     これは本です
freq         2
{'options': {'params': {'mode': 'stage', 'algo': 'transformers_zeroshot_classification', 'lang': 'jp', 'labels': '説明+歌詞+講演', 'prompt': '"この文は{}です"'}, 'feature_variables': ['text'], 'args': ['text'], 'model_name': 'transformers_zeroshot_classification', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['text']}


## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

Examples: | makeresults
| eval text="Demand and supply need to achieve equilibrium"
| fit MLTKContainer algo=transformers_zeroshot_classification lang=en labels=Economy+Ecology+Engineering prompt="This sentence is related to {}" from text into app:transformers_zeroshot_classification

In [16]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    try:
        if param['options']['params']['lang'] == "jp":
            pipe = pipeline("zero-shot-classification", model="Formzu/bert-base-japanese-jsnli")
        else:
            pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    except:
        pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    X = df.text.values
    candidate_labels = param['options']['params']['labels'].split("+")
    hypothesis_template = param['options']['params']['prompt']
    Y = []
    print(candidate_labels)
    print(hypothesis_template)
    for item in X:
        out = pipe(item, candidate_labels, hypothesis_template="This example is {}.")
        cand = out["labels"][0]
        cand_score = str(round(out["scores"][0], 2))
        Y.append(cand + " " + cand_score)

    cols={"label with score": Y}
    result=pd.DataFrame(data=cols)
    
    return result

In [15]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(None,df,param))

['説明', '歌詞', '講演']
"この文は{}です"
  label with score
0          歌詞 0.36
1          歌詞 0.36


## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code